In [10]:
import pandas as pd
import numpy as np
from src.utils import setup_folders, return_static_features, return_dynamic_features
from src.visualization.evaluate import calculate_metric_horizons, calculate_metric_horizon_windows, calculate_metric_horizons_all_models
from src.visualization.visualize import plot_horizons, plot_scatterplot, plot_test_day, plot_mean_std_error_multiple_models, plot_ridges_multiple_models
from src.features.build_features import build_features, build_features_LSTM
import json

In [11]:
def load_PICP_json(plant, model):
    # agr
    filename = f'../../reports/results/{plant}/probabilistic/distribution_simulation/final/{model}/0.295_0.35/bias_corrected/quantile_PICP_agr.json'
    f = open(filename)
    PICP_agr = json.load(f)
    f.close()

    # agr_sum
    filename = f'../../reports/results/{plant}/probabilistic/distribution_simulation/final/{model}/0.295_0.35/bias_corrected/quantile_PICP_agr_sum.json'
    f = open(filename)
    PICP_agr_sum = json.load(f)
    f.close()
    
    return(PICP_agr, PICP_agr_sum)

In [12]:
PICP_agr, PICP_agr_sum = load_PICP_json("HPP1", "LSTM")

In [17]:
# We add these to a dataframe with a multikey index
outer_index = ['Wind + Solar', 'Aggregated']
inner_index = ['LGB', 'LSTM', 'RF']
tuples = []
for outer in outer_index:
    for inner in inner_index:
        tuples.append((outer,inner))
        
index = pd.MultiIndex.from_tuples(tuples, names=["Method", "Model"])

In [33]:
plants = ["Nazeerabad", "HPP1", "HPP2", "HPP3"]
models = ['LGB', 'LSTM', 'RF']
def build_table(alpha = '0.95'):
    dict_results = {}
    for plant in plants:
        PICPs_agr = []
        PICPs_agr_sum = []
        for model in models:
            PICP_agr, PICP_agr_sum = load_PICP_json(plant, model)
            
            # Get the confidence level            
            PICPs_agr.append(PICP_agr[alpha])
            PICPs_agr_sum.append(PICP_agr_sum[alpha])
            
        # We append the lists
        PICPs = PICPs_agr + PICPs_agr_sum
        dict_results[plant] = np.round(PICPs,3)
        
    table = pd.DataFrame(dict_results, index = index)
    return(table)

In [36]:
table = build_table(alpha = '0.95')
table

Nazeerabad   HPP1   HPP2   HPP3
Method       Model                                 
Wind + Solar LGB         0.914  0.952  0.914  0.954
             LSTM        0.514  0.874  0.844  0.888
             RF          0.530  0.910  0.886  0.934
Aggregated   LGB         0.935  0.955  0.913  0.954
             LSTM        0.554  0.873  0.839  0.881
             RF          0.586  0.906  0.880  0.933

In [38]:
path_out_tables = f"../../reports/results/All Plants/Tables"
setup_folders(path_out_tables)

In [40]:
for alpha in ['0.95', '0.9', '0.75']:
    table = build_table(alpha)
    table.to_csv(f"{path_out_tables}/table_PICP_alpha_{alpha}.csv", sep = ";")